---

## 📚 Conceitos Fundamentais

### O que são Tokens? 🔤

**Tokens** são as unidades básicas de processamento de um modelo de linguagem (LLM).

**Analogia:** Imagine que você está lendo um livro em outro idioma. Você não lê letra por letra, mas sim **palavra por palavra** (ou às vezes sílabas). Para um LLM, os tokens são essas "palavras".

**Características:**
- 1 token ≈ 4 caracteres em português (varia conforme o modelo)
- Palavras comuns = 1 token (ex: "gato", "casa")
- Palavras raras = múltiplos tokens (ex: "extraordinário" = 3-4 tokens)
- Espaços e pontuação também são tokens

**Por que importam?**
- LLMs têm limite de tokens (ex: GPT-4 = 8k, 32k ou 128k tokens)
- APIs cobram por token processado
- Influenciam velocidade de processamento

---

### O que são Chunks? 📦

**Chunks** são pedaços de texto criados para **armazenamento e busca** em bancos vetoriais.

**Analogia:** Imagine que você está organizando uma biblioteca. Ao invés de guardar livros inteiros em uma prateleira (muito grande), você divide cada livro em **capítulos** (chunks). Assim, quando alguém pergunta "onde está a informação sobre fotossíntese?", você não precisa trazer o livro inteiro de biologia, apenas o capítulo relevante.

**Características:**
- Definidos em **caracteres** (ex: 1000 chars) ou **tokens** (ex: 256 tokens)
- Geralmente têm **overlap** (sobreposição) para não perder contexto
- Cada chunk vira **1 vetor** no banco vetorial

**Por que importam?**
- Buscas mais precisas (chunks menores = mais específicos)
- Controle de custo (menos tokens enviados ao LLM)
- Performance do banco vetorial (chunks balanceados = busca eficiente)

![Vectors chuck example](vectors_chunk_example.png)

---

## 🔍 A Diferença Crucial

| Aspecto | Tokens 🔤 | Chunks 📦 |
|---------|-----------|----------|
| **Definição** | Unidades de processamento do LLM | Pedaços de texto para busca |
| **Criado por** | Tokenizador do modelo (ex: tiktoken) | Você/desenvolvedor (splitter) |
| **Medido em** | Quantidade de subpalavras | Caracteres ou tokens |
| **Usado para** | Limites de contexto, custos de API | Indexação em banco vetorial |
| **Exemplo** | "extraordinário" = 3 tokens | "1000 caracteres de um PDF" = 1 chunk |
| **Importância em RAG** | Define quanto cabe no LLM | Define granularidade da busca |

### 💡 Exemplo Prático

Imagine um artigo científico de 10.000 caracteres:

**Chunking:**
- Você divide em 5 chunks de 2000 chars cada
- Cada chunk vira 1 embedding no FAISS/Qdrant
- Na busca, você recupera 2 chunks relevantes

**Tokenização:**
- Os 2 chunks recuperados (4000 chars) = ~1000 tokens
- Esses 1000 tokens são enviados ao LLM como contexto
- O LLM precisa caber isso + sua resposta no limite de tokens

**Em resumo:**
- **Chunks:** Como você organiza documentos para BUSCAR
- **Tokens:** Como o LLM LIDA com o que você encontrou

---

## 🛠️ Setup: Instalação de Bibliotecas

In [18]:
# Importações necessárias
import tiktoken
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd
from IPython.display import display, Markdown

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


---

## 🔢 Calculadora de Tokens

Vamos criar uma função para contar tokens usando `tiktoken`, a biblioteca oficial da OpenAI.

Um simulador pode ser visto em https://platform.openai.com/tokenizer

### 📖 O que é tiktoken?

`tiktoken` é um tokenizador criado pela OpenAI que mostra **exatamente** como modelos como GPT-3.5 e GPT-4 quebram texto em tokens.

**Encodings disponíveis:**
- `cl100k_base`: GPT-4, GPT-3.5-turbo, text-embedding-ada-002
- `p50k_base`: Codex, text-davinci-002, text-davinci-003
- `r50k_base`: GPT-3 (davinci, curie, babbage, ada)

In [19]:
def calc_tokens(text: str, encoding_name: str = "cl100k_base") -> dict:
    """
    Calcula tokens de um texto usando tiktoken.
    
    Args:
        text: Texto para tokenizar
        encoding_name: Nome do encoding (padrão: cl100k_base para GPT-4)
    
    Returns:
        Dict com estatísticas de tokens
    """
    # Carrega o encoding
    encoding = tiktoken.get_encoding(encoding_name)
    
    # Tokeniza
    tokens = encoding.encode(text)
    
    # Calcula estatísticas
    num_chars = len(text)
    num_tokens = len(tokens)
    ratio = num_chars / num_tokens if num_tokens > 0 else 0
    
    return {
        "texto": text[:100] + "..." if len(text) > 100 else text,
        "caracteres": num_chars,
        "tokens": num_tokens,
        "chars_por_token": round(ratio, 2),
        "encoding": encoding_name,
        "tokens_list": tokens[:20]  # Primeiros 20 tokens para inspeção
    }

# Função para exibir de forma bonita
def show_token_counts(results: dict):
    """Exibe resultado da contagem de tokens de forma formatada."""
    print("\n" + "="*80)
    print("📊 ANÁLISE DE TOKENS")
    print("="*80)
    print(f"\n📝 Texto (preview): {results['texto']}")
    print(f"\n📏 Caracteres: {results['caracteres']:,}")
    print(f"🔢 Tokens: {results['tokens']:,}")
    print(f"📐 Proporção: {results['chars_por_token']} chars/token")
    print(f"⚙️  Encoding: {results['encoding']}")
    print(f"\n🔍 Primeiros 20 IDs de tokens: {results['tokens_list']}")
    print("="*80 + "\n")

print("✅ Calculadora de tokens criada!")

✅ Calculadora de tokens criada!


### 🧪 Teste 1: Frase Simples

In [20]:
# Teste com frase simples
text = "Olá, meu nome é João e eu estudo banco de dados vetoriais!"

results = calc_tokens(text)
show_token_counts(results)

# Insight
print("💡 INSIGHT:")
print(f"   Uma frase de {results['caracteres']} caracteres = {results['tokens']} tokens")
print(f"   Proporção: ~{results['chars_por_token']} caracteres por token")


📊 ANÁLISE DE TOKENS

📝 Texto (preview): Olá, meu nome é João e eu estudo banco de dados vetoriais!

📏 Caracteres: 58
🔢 Tokens: 19
📐 Proporção: 3.05 chars/token
⚙️  Encoding: cl100k_base

🔍 Primeiros 20 IDs de tokens: [43819, 1995, 11, 56309, 17567, 4046, 11186, 3496, 384, 15925, 1826, 7835, 53565, 409, 29045, 24195, 11015, 285, 0]

💡 INSIGHT:
   Uma frase de 58 caracteres = 19 tokens
   Proporção: ~3.05 caracteres por token


### 🧪 Teste 2: Texto Técnico

**Observação**: Os tokenizadores genéricos como o `cl100k_base` da OpenAI não são otimizados para português, resultando em mais tokens necessários para representar o mesmo texto. O BERTugues, ao remover caracteres irrelevantes para português, consegue reduzir significativamente o tamanho da representação, economizando no processamento e custos de API

In [21]:
# Teste com texto técnico (típico de RAG)
text_pt = """
Retrieval-Augmented Generation (RAG) é uma arquitetura que combina recuperação de informações 
com geração de linguagem natural. O sistema funciona em duas etapas: primeiro, um retriever 
busca documentos relevantes em um banco vetorial usando embeddings semânticos; depois, um 
Large Language Model (LLM) utiliza esses documentos como contexto para gerar respostas precisas 
e fundamentadas. A vantagem principal do RAG é a capacidade de acessar conhecimento externo sem 
necessidade de fine-tuning do modelo.
"""

text_en = """
Retrieval-Augmented Generation (RAG) is an architecture that combines information retrieval
with natural language generation. The system operates in two stages: first, a retriever
searches for relevant documents in a vector database using semantic embeddings; then, a
Large Language Model (LLM) uses these documents as context to generate accurate and
grounded responses. The main advantage of RAG is the ability to access external knowledge
without the need for fine-tuning the model.
"""


In [22]:

tokenization_result_pt = calc_tokens(text_pt)
tokenization_result_en = calc_tokens(text_en)
show_token_counts(tokenization_result_en)
show_token_counts(tokenization_result_pt)



📊 ANÁLISE DE TOKENS

📝 Texto (preview): 
Retrieval-Augmented Generation (RAG) is an architecture that combines information retrieval
with na...

📏 Caracteres: 487
🔢 Tokens: 99
📐 Proporção: 4.92 chars/token
⚙️  Encoding: cl100k_base

🔍 Primeiros 20 IDs de tokens: [198, 12289, 7379, 838, 62735, 28078, 24367, 320, 49, 1929, 8, 374, 459, 18112, 430, 33511, 2038, 57470, 198, 4291]


📊 ANÁLISE DE TOKENS

📝 Texto (preview): 
Retrieval-Augmented Generation (RAG) é uma arquitetura que combina recuperação de informações 
com ...

📏 Caracteres: 512
🔢 Tokens: 125
📐 Proporção: 4.1 chars/token
⚙️  Encoding: cl100k_base

🔍 Primeiros 20 IDs de tokens: [198, 12289, 7379, 838, 62735, 28078, 24367, 320, 49, 1929, 8, 4046, 10832, 802, 32637, 295, 5808, 1744, 3698, 2259]



In [23]:

# Comparação com diferentes encodings
print("\n🔬 COMPARAÇÃO DE ENCODINGS:\n")
encodings = [
    "cl100k_base",  # Usado na família GPT‑3.5 e GPT‑4 (por exemplo, gpt‑3.5‑turbo e gpt‑4)
    "p50k_base",    # Foi usado em modelos GPT‑3 mais novos (e em Codex), com vocabulário de 50k tokens.

    ]
encoding_comparison = []

for enc in encodings:
    res = calc_tokens(text_en, enc)
    encoding_comparison.append({
        "Encoding": enc,
        "Usado em": "GPT-4" if enc == "cl100k_base" else "GPT-3",
        "Tokens": res['tokens'],
        "Chars/Token": res['chars_por_token']
    })

df_encodings = pd.DataFrame(encoding_comparison)
display(df_encodings)

print("\n💡 INSIGHT:")
print("   Diferentes modelos tokenizam de forma diferente!")
print("   Sempre use o encoding correspondente ao seu modelo.")


🔬 COMPARAÇÃO DE ENCODINGS:



,Encoding,Usado em,Tokens,Chars/Token
0,cl100k_base,GPT-4,99,4.92
1,p50k_base,GPT-3,104,4.68



💡 INSIGHT:
   Diferentes modelos tokenizam de forma diferente!
   Sempre use o encoding correspondente ao seu modelo.


---

## Calculadora de Chunks

Agora vamos criar uma calculadora de chunks que mostra como diferentes configurações afetam a divisão do texto.

### Parâmetros Principais do Chunking

1. **chunk_size**: Tamanho máximo de cada chunk (em caracteres)
2. **chunk_overlap**: Quantos caracteres de sobreposição entre chunks consecutivos
3. **separators**: Ordem de prioridade para quebrar o texto (ex: parágrafos → linhas → palavras)

**Por que overlap é importante?**
- Evita cortar frases/conceitos importantes no meio
- Mantém contexto entre chunks adjacentes
- Geralmente usa-se 10-20% do chunk_size

**Como funciona os separadores**

Os **separadores** definem a **ordem de prioridade** de onde o texto pode ser quebrado ao criar chunks. É uma lista hierárquica que o `RecursiveCharacterTextSplitter` usa para dividir o texto de forma inteligente.

O algoritmo tenta **primeiro** usar o separador mais "natural" e, se não conseguir respeitar o `chunk_size`, vai descendo na hierarquia:

```python
separators = ["\n\n", "\n", " ", ""]
```





**Ordem de tentativa:**

1. **`\n\n`** (duas quebras de linha) → Quebra por **parágrafos**
   - Mais natural, mantém ideias completas juntas
   - Ideal para textos estruturados

2. **`\n`** (uma quebra de linha) → Quebra por **linhas**
   - Se parágrafos forem muito grandes
   - Útil para listas, código, etc.

3. **` `** (espaço) → Quebra por **palavras**
   - Se linhas forem muito longas
   - Evita cortar palavras no meio

4. **`""`** (vazio) → Quebra em **qualquer caractere**
   - Último recurso
   - Quando nem palavras cabem (muito raro)

**💡 Exemplo prático:**

Imagine este texto:

```text
Parágrafo 1: Este é um texto longo sobre RAG.\n
Aqui continua a explicação sobre retrieval.\n\n

Parágrafo 2: Agora vou falar de embeddings.\n
Embeddings são representações vetoriais.
```




Com `chunk_size=100` e `separators=["\n\n", "\n", " ", ""]`:

- **1ª tentativa:** Quebrar em `\n\n` → Se parágrafo 1 tiver < 100 chars, fica inteiro no chunk
- **Se não couber:** Quebra em `\n` → Divide por linhas dentro do parágrafo
- **Se ainda não couber:** Quebra em ` ` → Divide por palavras
- **Último caso:** Quebra caractere por caractere

✅ Por que isso é importante?

Mantém a **coerência semântica** dos chunks:
- ✅ "A fotossíntese é um processo..." (chunk completo)
- ❌ "A fotossín..." (cortado no meio - péssimo para embeddings!)

In [24]:
def calc_chunks(text: str, 
                chunk_size: int = 1000, 
                chunk_overlap: int = 200,
                separators: list = []) -> dict:
    """
    Calcula chunks de um texto usando RecursiveCharacterTextSplitter.
    
    Args:
        texto: Texto para dividir
        chunk_size: Tamanho máximo de cada chunk
        chunk_overlap: Sobreposição entre chunks
        separators: Lista de separadores (padrão: parágrafos, linhas, espaços)
    
    Returns:
        Dict com estatísticas de chunking
    """
    if not separators:
        separators = ["\n\n", "\n", " ", ""]
    
    # Cria o splitter
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=separators
    )
    
    # Divide o texto
    chunks = splitter.split_text(text)
    
    # Estatísticas
    chunk_sizes = [len(c) for c in chunks]
    
    return {
        "texto_original": text,
        "chunks": chunks,
        "num_chunks": len(chunks),
        "chunk_size_config": chunk_size,
        "chunk_overlap_config": chunk_overlap,
        "tamanho_medio": sum(chunk_sizes) / len(chunk_sizes) if chunk_sizes else 0,
        "tamanho_min": min(chunk_sizes) if chunk_sizes else 0,
        "tamanho_max": max(chunk_sizes) if chunk_sizes else 0,
        "chars_totais": len(text),
        "separators": separators
    }

def show_chunk_statistics(results: dict, display_content: bool = True):
    """Exibe resultado da divisão em chunks de forma formatada."""
    print("\n" + "="*80)
    print("📦 ANÁLISE DE CHUNKS")
    print("="*80)
    
    print(f"\n⚙️  CONFIGURAÇÃO:")
    print(f"   Chunk Size: {results['chunk_size_config']} chars")
    print(f"   Chunk Overlap: {results['chunk_overlap_config']} chars ({results['chunk_overlap_config']/results['chunk_size_config']*100:.0f}%)")
    print(f"   Separators: {results['separators']}")
    
    print(f"\n📊 ESTATÍSTICAS:")
    print(f"   Caracteres totais: {results['chars_totais']:,}")
    print(f"   Número de chunks: {results['num_chunks']}")
    print(f"   Tamanho médio: {results['tamanho_medio']:.0f} chars")
    print(f"   Tamanho mínimo: {results['tamanho_min']} chars")
    print(f"   Tamanho máximo: {results['tamanho_max']} chars")
    
    if display_content:
        print(f"\n📝 CONTEÚDO DOS CHUNKS:\n")
        for i, chunk in enumerate(results['chunks'], 1):
            print(f"\n{'─'*80}")
            print(f"Chunk {i}/{results['num_chunks']} ({len(chunk)} chars)")
            print(f"{'─'*80}")
            preview = chunk[:300] + "..." if len(chunk) > 300 else chunk
            print(preview)
    
    print("\n" + "="*80 + "\n")

print("✅ Calculadora de chunks criada!")

✅ Calculadora de chunks criada!


### 🧪 Teste 3: Texto Médio com Diferentes Configurações

In [35]:
# Texto de exemplo (simulando um artigo)
article = """
Introdução aos Bancos de Dados Vetoriais

Os bancos de dados vetoriais são uma tecnologia emergente crucial para aplicações de inteligência 
artificial moderna. Eles armazenam dados como vetores de alta dimensão (embeddings) e permitem 
buscas por similaridade semântica.

Principais Características

Um embedding é uma representação numérica de dados (texto, imagens, áudio) em um espaço vetorial. 
Por exemplo, a frase "gato" pode ser representada como um vetor de 768 dimensões. Frases com 
significados similares terão vetores próximos no espaço vetorial.

Aplicações Práticas

Sistemas RAG (Retrieval-Augmented Generation) são a aplicação mais popular. Eles combinam busca 
vetorial com modelos de linguagem para criar assistentes que respondem com base em documentos 
específicos. Outras aplicações incluem sistemas de recomendação, busca semântica em e-commerce, 
e detecção de anomalias.

Tecnologias Disponíveis

As principais soluções incluem FAISS (Facebook AI), Qdrant, Pinecone, Weaviate e Milvus. FAISS 
é ideal para prototipagem local, enquanto Qdrant oferece recursos de produção com API REST completa. 
Pinecone é uma solução gerenciada na nuvem, e Weaviate oferece suporte nativo a GraphQL.

Desafios e Considerações

O principal desafio é o chunking adequado dos documentos. Chunks muito grandes perdem precisão na 
busca, enquanto chunks muito pequenos perdem contexto. O overlap entre chunks ajuda a manter a 
coerência semântica. Outra consideração importante é a escolha do modelo de embeddings, que deve 
ser consistente entre indexação e busca.
"""

print(f"📄 Texto de exemplo: {len(article)} caracteres\n")

# Teste com configuração padrão
chunked_article_results = calc_chunks(article, chunk_size=500, chunk_overlap=100)
show_chunk_statistics(chunked_article_results, display_content=False)

📄 Texto de exemplo: 1571 caracteres


📦 ANÁLISE DE CHUNKS

⚙️  CONFIGURAÇÃO:
   Chunk Size: 500 chars
   Chunk Overlap: 100 chars (20%)
   Separators: ['\n\n', '\n', ' ', '']

📊 ESTATÍSTICAS:
   Caracteres totais: 1,571
   Número de chunks: 5
   Tamanho médio: 332 chars
   Tamanho mínimo: 299 chars
   Tamanho máximo: 359 chars




### 🧪 Teste 4: Comparação de Diferentes Chunk Sizes

In [36]:
# Testar diferentes tamanhos de chunk
configs = [
    {"chunk_size": 200, "chunk_overlap": 40},   # Chunks pequenos
    {"chunk_size": 500, "chunk_overlap": 100},  # Chunks médios
    {"chunk_size": 1000, "chunk_overlap": 200}, # Chunks grandes
]

print("\n🔬 COMPARAÇÃO DE CONFIGURAÇÕES DE CHUNKING\n")

chunking_comparison_results = []

for config in configs:
    res = calc_chunks(article, **config)
    chunking_comparison_results.append({
        "Chunk Size": config['chunk_size'],
        "Overlap": config['chunk_overlap'],
        "Overlap %": f"{config['chunk_overlap']/config['chunk_size']*100:.0f}%",
        "Nº Chunks": res['num_chunks'],
        "Tamanho Médio": f"{res['tamanho_medio']:.0f} chars",
        "Tamanho Min": res['tamanho_min'],
        "Tamanho Max": res['tamanho_max']
    })

df_chunks = pd.DataFrame(chunking_comparison_results)
display(df_chunks)

print("\n💡 INSIGHTS:")
print("   - Chunks menores = mais chunks = busca mais precisa (mas menos contexto)")
print("   - Chunks maiores = menos chunks = mais contexto (mas busca menos precisa)")
print("   - Overlap de 20% (do chunk_size) é um bom equilíbrio")
print("   - Para produção RAG, chunk_size entre 500-1000 chars é comum")


🔬 COMPARAÇÃO DE CONFIGURAÇÕES DE CHUNKING



,Chunk Size,Overlap,Overlap %,Nº Chunks,Tamanho Médio,Tamanho Min,Tamanho Max
0,200,40,20%,15,103 chars,19,196
1,500,100,20%,5,332 chars,299,359
2,1000,200,20%,2,796 chars,672,920



💡 INSIGHTS:
   - Chunks menores = mais chunks = busca mais precisa (mas menos contexto)
   - Chunks maiores = menos chunks = mais contexto (mas busca menos precisa)
   - Overlap de 20% (do chunk_size) é um bom equilíbrio
   - Para produção RAG, chunk_size entre 500-1000 chars é comum


## Importância do Tamanho do Chunk

O tamanho do chunk é **crucial** porque afeta diretamente três aspectos do sistema RAG:

### 1. Precisão da Busca Semântica

**Chunks pequenos (200-400 chars):**
- ✅ Alta precisão: retornam informação muito específica
- ❌ Pouco contexto: podem perder o "quadro geral"
- 📍 Uso: FAQs, dicionários, documentação de APIs

**Chunks grandes (1500-2000 chars):**
- ✅ Muito contexto: capturam conceitos completos
- ❌ Baixa precisão: diluem a relevância semântica
- 📍 Uso: narrativas, análises longas

### 2. Custo e Performance

- **Custo de API:** Mais chunks recuperados (k alto) com chunks grandes = mais tokens = mais caro
- **Latência:** Chunks grandes demoram mais para gerar embeddings e retornar
- **Armazenamento:** Cada chunk = 1 vetor no banco (chunks pequenos = mais vetores)

### 3. Qualidade das Respostas do LLM

- **Context window:** LLMs têm limite de tokens (ex: GPT-4 = 8k, 128k)
- **Chunk muito grande:** Pode estourar o limite mesmo com k=2 ou k=3
- **Chunk muito pequeno:** LLM recebe informação fragmentada, gera respostas incompletas

---

## Tamanhos Padrão da Indústria

### Configurações mais comuns (2024-2025)


| Aplicação | Chunk Size (chars) | Chunk Size (tokens) | Overlap | k | Justificativa |
|-----------|-------------------|---------------------|---------|---|---------------|
| **RAG Genérico (Baseline)** | **1600-2000** | **400-512** | **10-20%** | **4-5** | Sweet spot recomendado; equilíbrio ideal entre contexto e precisão |
| **FAQ/Suporte** | **800-1200** | **200-400** | **10-15%** | **4-6** | Respostas isoladas e completas; questões específicas |
| **Chatbots Empresariais** | **1600-2400** | **400-600** | **15-20%** | **4-5** | Contexto mais amplo para respostas complexas; não tão pequeno quanto FAQ |
| **Documentação Técnica** | **2000-2800** | **500-700** | **20-25%** | **3-4** | Inclui exemplos, parâmetros e contexto completo de métodos API |
| **Análise de Documentos** | **2400-3600** | **600-900** | **20-25%** | **2-4** | Contexto profundo para análise holística; busca semântica |
| **Academic Papers** | **2800-4800** | **700-1200** | **30%+** | **2-3** | Construção incremental de conceitos; preservação de figuras e tabelas |
| **Legal/Compliance** | **3200-5000** | **800-1250** | **25-30%** | **2-3** | Precisão jurídica; mantém cláusulas e referências legais completas |
| **Code Search/Repositories** | **1200-2000** | **300-512** | **10-15%** | **3-5** | Funções e classes completas; respeita limites de estrutura de código |
| **News/Blog Posts** | **1600-2400** | **400-600** | **15-20%** | **4-5** | Estrutura editorial já otimizada para leitura; parágrafos naturais |

#### Notas Importantes

- **Sweet Spot**: **256-512 tokens** é o intervalo recomendado pela maioria dos especialistas como ponto de partida para experimentação
- **Overlap**: recomendável **10-20% de overlap** para a maioria dos casos, aumentando para **25-30%** em documentos legais e acadêmicos para preservar contexto.
- **Parâmetro k** (chunks recuperados): sugere-se **k=5** como padrão em muitos estudos de RAG, com variação de **2-6** conforme a aplicação. Valores mais baixos (2-3) para documentos densos (legal, academic); valores mais altos (4-6) para consultas gerais
- **Estratégia de Chunking**: os valores acima assumem **recursive/semantic chunking** (padrão recomendado para 80% das aplicações), não simple size-based chunking.

#### Referências da Indústria

- https://www.pinecone.io/learn/rag-best-practices/#chunk-size
- https://www.firecrawl.dev/blog/best-chunking-strategies-rag-2025
- https://arxiv.org/html/2508.02435v1
- https://www.sciencedirect.com/science/article/pii/S0920548925000248

### Conversão Aproximada

**Regra prática:** 
- **1 token ≈ 4 caracteres** (português)
- **1 token ≈ 4 caracteres** (inglês)
- **Chunk de 1000 chars ≈ 250 tokens**
- **Chunk de 1800 chars ≈ 450 tokens** (baseline 2024-2025)
- **Chunk de 2000 chars ≈ 512 tokens** (limite superior baseline)

## Recomendação da Indústria (Consenso 2024-2025)

### **Sweet Spot Universal:**

```python
chunk_size = 1800        # caracteres (baseline: 1600-2000)
chunk_overlap = 300      # ~17% do chunk_size (range: 10-20%)
k = 4                    # chunks recuperados (range: 4-5)
chunk_overlap = 200      # 20% do chunk_size
```


**📊 Evolução das Recomendações:**
- **2024-2025:** 1600-2000 chars (sweet spot: 1800)
- **2022-2023:** 800-1000 chars era o padrão



**Por quê?**
1. **1800 chars ≈ 450 tokens/chunk** → Equilíbrio ideal entre contexto e precisão
2. **k=4 chunks = ~1800 tokens** → Contexto rico sem estourar limites de modelos modernos
3. **Overlap de 17% (300 chars)** → Preserva contexto entre chunks sem duplicação excessiva
4. **Flexível:** Funciona bem para 70-80% dos casos de uso genéricos
5. **Compatível:** Aproveitam context windows maiores dos LLMs modernos (GPT-4: 128k, Claude-3: 200k)

---

## 📐 Como Escolher o Tamanho Ideal?

### Método Empírico (recomendado para produção):
1. **Comece com 1800 chars, overlap 300** (baseline 2024-2025)
2. **Teste com 20-50 perguntas reais** do seu domínio
3. **Meça métricas:**
   - Relevância das respostas (1-5)
   - Taxa de "não sei" (se LLM não acha informação)
   - Latência de resposta
4. **Ajuste iterativamente:**
   - Respostas vagas/imprecisas? → Reduza chunk_size (1200-1400)
   - Respostas fragmentadas? → Aumente chunk_size (2200-2400)
   - Custo alto? → Reduza k (de 4 para 3) ou chunk_size
   - Performance lenta? → Reduza chunk_size ou k

### 🧪 Exemplo de Teste A/B:

```python

test_configs = [
    {"chunk_size": 1200, "overlap": 200, "k": 5},   # Precisão (FAQ/Suporte)
    {"chunk_size": 1800, "overlap": 300, "k": 4},   # Baseline 2024-2025
    {"chunk_size": 2400, "overlap": 400, "k": 3},   # Contexto (Docs Técnicos)
    {"chunk_size": 1000, "overlap": 200, "k": 4},  # Padrão
    {"chunk_size": 1500, "overlap": 300, "k": 3},  # Contexto
]
```

- Teste com perguntas reais do seu domínio
- Meça qualidade, custo e latência
- Compare com métricas de baseline





---

## Erros Comuns

❌ **"Vou usar 5000 chars para ter mais contexto!"**
→ Resultado: Busca imprecisa, retorna chunks irrelevantes

❌ **"Vou usar 100 chars para ter precisão máxima!"**
→ Resultado: Respostas fragmentadas, LLM não entende contexto

❌ **"Vou usar k=10 para garantir que tenho tudo!"**
→ Resultado: Custo alto, informação irrelevante dilui a relevante

✅ **Abordagem correta:**
- Comece com padrão da indústria (1800/300/k=4)
- Meça resultados com dados reais
- Ajuste baseado em métricas objetivas



---

## 🎯 Calculadora Combinada: Tokens + Chunks

Agora vamos combinar as duas análises para entender o impacto completo no sistema RAG.

In [37]:
def perform_rag_analysis(text: str,
                        chunk_size: int = 1000,
                        chunk_overlap: int = 200,
                        num_chunks_retrieved: int = 4,
                        encoding: str = "cl100k_base"):
    """
    Análise completa simulando um pipeline RAG:
    1. Divide o texto em chunks
    2. Calcula tokens por chunk
    3. Simula recuperação de k chunks
    4. Calcula tokens totais que serão enviados ao LLM
    
    Args:
        texto: Documento original
        chunk_size: Tamanho dos chunks
        chunk_overlap: Overlap entre chunks
        k_recuperados: Quantos chunks serão recuperados na busca
        encoding: Encoding do tiktoken
    """
    print("\n" + "="*80)
    print("🎯 ANÁLISE COMPLETA RAG: CHUNKS + TOKENS")
    print("="*80)
    
    # 1. Análise do documento original
    print("\n📄 DOCUMENTO ORIGINAL:")
    tokens_original = calc_tokens(text, encoding)
    print(f"   Caracteres: {tokens_original['caracteres']:,}")
    print(f"   Tokens: {tokens_original['tokens']:,}")
    
    # 2. Chunking
    print("\n✂️  CHUNKING:")
    chunks_result = calc_chunks(text, chunk_size, chunk_overlap)
    print(f"   Configuração: {chunk_size} chars, overlap {chunk_overlap} ({chunk_overlap/chunk_size*100:.0f}%)")
    print(f"   Chunks criados: {chunks_result['num_chunks']}")
    print(f"   Tamanho médio: {chunks_result['tamanho_medio']:.0f} chars/chunk")
    
    # 3. Tokens por chunk
    print("\n🔢 TOKENS POR CHUNK:")
    tokens_por_chunk = []
    for i, chunk in enumerate(chunks_result['chunks'], 1):
        tokens_chunk = calc_tokens(chunk, encoding)
        tokens_por_chunk.append({
            "Chunk": i,
            "Caracteres": len(chunk),
            "Tokens": tokens_chunk['tokens']
        })
    
    df_tokens_chunks = pd.DataFrame(tokens_por_chunk)
    display(df_tokens_chunks)
    
    # 4. Simulação de recuperação (RAG)
    print(f"\n🔍 SIMULAÇÃO DE BUSCA (k={num_chunks_retrieved}):")
    chunks_recuperados = min(num_chunks_retrieved, chunks_result['num_chunks'])
    tokens_recuperados = sum(tokens_por_chunk[i]['Tokens'] for i in range(chunks_recuperados))
    chars_recuperados = sum(tokens_por_chunk[i]['Caracteres'] for i in range(chunks_recuperados))
    
    print(f"   Chunks recuperados: {chunks_recuperados}")
    print(f"   Caracteres enviados ao LLM: {chars_recuperados:,}")
    print(f"   Tokens enviados ao LLM: {tokens_recuperados:,}")
    
    # 5. Análise de custo/eficiência
    print("\n💰 ANÁLISE DE EFICIÊNCIA:")
    reducao_chars = (1 - chars_recuperados / tokens_original['caracteres']) * 100
    reducao_tokens = (1 - tokens_recuperados / tokens_original['tokens']) * 100
    
    print(f"   Redução de caracteres: {reducao_chars:.1f}%")
    print(f"   Redução de tokens: {reducao_tokens:.1f}%")
    print(f"   Economia: Você está enviando apenas {100-reducao_tokens:.1f}% do documento!")
    
    # 6. Limites de contexto
    print("\n🚦 VERIFICAÇÃO DE LIMITES:")
    limites = {
        "GPT-3.5-turbo": 4096,
        "GPT-4": 8192,
        "GPT-4-turbo": 128000,
        "Claude-3": 200000
    }
    
    # Reservar 500 tokens para resposta
    tokens_com_resposta = tokens_recuperados + 500
    
    for modelo, limite in limites.items():
        percentual = (tokens_com_resposta / limite) * 100
        status = "✅" if percentual < 80 else "⚠️" if percentual < 100 else "❌"
        print(f"   {status} {modelo} ({limite:,} tokens): {percentual:.1f}% usado")
    
    print("\n" + "="*80 + "\n")
    
    return {
        "chunks_result": chunks_result,
        "tokens_original": tokens_original,
        "tokens_recuperados": tokens_recuperados,
        "chars_recuperados": chars_recuperados
    }

print("✅ Calculadora combinada criada!")

✅ Calculadora combinada criada!


### 🧪 Teste 5: Análise Completa do Artigo

In [38]:
# Análise completa com configuração típica de RAG
complete_analysis_result = perform_rag_analysis(
    text=article,
    chunk_size=500,
    chunk_overlap=100,
    num_chunks_retrieved=3,
    encoding="cl100k_base"
)


🎯 ANÁLISE COMPLETA RAG: CHUNKS + TOKENS

📄 DOCUMENTO ORIGINAL:
   Caracteres: 1,571
   Tokens: 418

✂️  CHUNKING:
   Configuração: 500 chars, overlap 100 (20%)
   Chunks criados: 5
   Tamanho médio: 332 chars/chunk

🔢 TOKENS POR CHUNK:


,Chunk,Caracteres,Tokens
0,1,299,79
1,2,307,81
2,3,359,94
3,4,337,99
4,5,359,88



🔍 SIMULAÇÃO DE BUSCA (k=3):
   Chunks recuperados: 3
   Caracteres enviados ao LLM: 965
   Tokens enviados ao LLM: 254

💰 ANÁLISE DE EFICIÊNCIA:
   Redução de caracteres: 38.6%
   Redução de tokens: 39.2%
   Economia: Você está enviando apenas 60.8% do documento!

🚦 VERIFICAÇÃO DE LIMITES:
   ✅ GPT-3.5-turbo (4,096 tokens): 18.4% usado
   ✅ GPT-4 (8,192 tokens): 9.2% usado
   ✅ GPT-4-turbo (128,000 tokens): 0.6% usado
   ✅ Claude-3 (200,000 tokens): 0.4% usado




---

## 🇧🇷 Teste 6: Tokenização Otimizada para Português

Vamos comparar a tokenização do `tiktoken` (otimizado para inglês) com o **BERTugues** (otimizado para português).

### 📖 O que é BERTugues?

BERTugues é uma versão do BERT treinada especificamente para português brasileiro. Sua principal vantagem é a **eficiência na tokenização** de textos em português, resultando em menos tokens para representar o mesmo conteúdo.

In [39]:
# Importações para tokenização com BERTugues
from transformers import BertTokenizer, BertModel
import torch

print("✅ Bibliotecas para BERTugues importadas!")

✅ Bibliotecas para BERTugues importadas!


In [40]:
# Carregar o tokenizador e o modelo BERTugues
print("🔄 Carregando BERTugues (pode levar alguns segundos)...\n")

tokenizer_bert = BertTokenizer.from_pretrained(
    "ricardoz/BERTugues-base-portuguese-cased", 
    do_lower_case=False
)

model_bert = BertModel.from_pretrained(
    "ricardoz/BERTugues-base-portuguese-cased"
)

print("✅ BERTugues carregado com sucesso!")
print(f"📊 Vocabulário: {tokenizer_bert.vocab_size:,} tokens")

🔄 Carregando BERTugues (pode levar alguns segundos)...

✅ BERTugues carregado com sucesso!
📊 Vocabulário: 30,522 tokens


In [41]:
def calc_tokens_bertugues(text: str) -> dict:
    """
    Calcula tokens usando BERTugues (otimizado para português).
    
    Args:
        text: Texto para tokenizar
    
    Returns:
        Dict com estatísticas de tokens e embeddings
    """
    # Tokenizar
    tokens = tokenizer_bert.tokenize(text)
    input_ids = tokenizer_bert.encode(text, return_tensors='pt')
    
    # Gerar embeddings
    with torch.no_grad():
        outputs = model_bert(input_ids)
        embeddings = outputs.last_hidden_state
    
    # Calcular estatísticas
    num_chars = len(text)
    num_tokens = len(tokens)
    ratio = num_chars / num_tokens if num_tokens > 0 else 0
    
    return {
        "texto": text[:100] + "..." if len(text) > 100 else text,
        "caracteres": num_chars,
        "tokens": num_tokens,
        "chars_por_token": round(ratio, 2),
        "encoding": "BERTugues",
        "tokens_list": tokens[:20],  # Primeiros 20 tokens
        "embedding_shape": embeddings.shape,
        "vocab_size": tokenizer_bert.vocab_size
    }

# Teste com o mesmo texto técnico usado anteriormente
resultado_bert = calc_tokens_bertugues(text_pt)

# Exibir usando formato similar aos testes anteriores
print("\n" + "="*80)
print("📊 ANÁLISE DE TOKENS - BERTugues")
print("="*80)
print(f"\n📝 Texto (preview): {resultado_bert['texto']}")
print(f"\n📏 Caracteres: {resultado_bert['caracteres']:,}")
print(f"🔢 Tokens: {resultado_bert['tokens']:,}")
print(f"📐 Proporção: {resultado_bert['chars_por_token']} chars/token")
print(f"⚙️  Encoding: {resultado_bert['encoding']}")
print(f"📦 Tamanho do vocabulário: {resultado_bert['vocab_size']:,}")
print(f"🧮 Shape dos embeddings: {resultado_bert['embedding_shape']}")
print(f"\n🔍 Primeiros 20 tokens:")
for i in range(0, min(20, len(resultado_bert['tokens_list'])), 5):
    tokens_linha = resultado_bert['tokens_list'][i:i+5]
    print(f"   {tokens_linha}")
print("="*80 + "\n")

# Comparação com tiktoken
print("🔬 COMPARAÇÃO: BERTugues vs tiktoken (cl100k_base)\n")

comparacao_modelos = pd.DataFrame([
    {
        "Modelo": "tiktoken (GPT-4)",
        "Otimizado para": "Inglês",
        "Tokens": tokenization_result_pt['tokens'],
        "Chars/Token": tokenization_result_pt['chars_por_token'],
        "Economia": "baseline"
    },
    {
        "Modelo": "BERTugues",
        "Otimizado para": "Português",
        "Tokens": resultado_bert['tokens'],
        "Chars/Token": resultado_bert['chars_por_token'],
        "Economia": f"{((tokenization_result_pt['tokens'] - resultado_bert['tokens']) / tokenization_result_pt['tokens'] * 100):.1f}%"
    }
])

display(comparacao_modelos)

print("\n💡 INSIGHTS:")
print(f"   - BERTugues usa {resultado_bert['tokens']} tokens vs {tokenization_result_pt['tokens']} do tiktoken")
print(f"   - Redução de {((tokenization_result_pt['tokens'] - resultado_bert['tokens']) / tokenization_result_pt['tokens'] * 100):.1f}% no número de tokens")
print(f"   - Cada token do BERTugues representa ~{resultado_bert['chars_por_token']} caracteres")
print("   - Para textos em português, BERTugues é mais eficiente!")
print("   - Economia direta em custo de API e processamento")


📊 ANÁLISE DE TOKENS - BERTugues

📝 Texto (preview): 
Retrieval-Augmented Generation (RAG) é uma arquitetura que combina recuperação de informações 
com ...

📏 Caracteres: 512
🔢 Tokens: 105
📐 Proporção: 4.88 chars/token
⚙️  Encoding: BERTugues
📦 Tamanho do vocabulário: 30,522
🧮 Shape dos embeddings: torch.Size([1, 107, 768])

🔍 Primeiros 20 tokens:
   ['Ret', '##rie', '##val', '-', 'Aug']
   ['##mente', '##d', 'Generation', '(', 'RA']
   ['##G', ')', 'é', 'uma', 'arquitetura']
   ['que', 'combina', 'recuperação', 'de', 'informações']

🔬 COMPARAÇÃO: BERTugues vs tiktoken (cl100k_base)



,Modelo,Otimizado para,Tokens,Chars/Token,Economia
0,tiktoken (GPT-4),Inglês,125,4.10,baseline
1,BERTugues,Português,105,4.88,16.0%



💡 INSIGHTS:
   - BERTugues usa 105 tokens vs 125 do tiktoken
   - Redução de 16.0% no número de tokens
   - Cada token do BERTugues representa ~4.88 caracteres
   - Para textos em português, BERTugues é mais eficiente!
   - Economia direta em custo de API e processamento


---

## 🎓 Exercícios Práticos

Agora é sua vez! Use as calculadoras acima para explorar diferentes cenários.

### 📝 Exercício 1: Otimização de Chunks

**Desafio:** Você tem um documento de 5000 caracteres e quer recuperar 4 chunks. Qual configuração de `chunk_size` e `chunk_overlap` resulta em:
- Aproximadamente 6-8 chunks totais?
- Tokens enviados ao LLM < 1000?

**Instruções:**
1. Crie um texto de exemplo de ~5000 chars
2. Teste diferentes configurações
3. Use `analise_completa_rag()` para verificar

In [42]:
# SEU CÓDIGO AQUI
# Dica: Comece com chunk_size=800, chunk_overlap=160

# texto_exercicio_1 = "..." * 100  # crie um texto de ~5000 chars
# analise_completa_rag(texto_exercicio_1, chunk_size=???, chunk_overlap=???, k_recuperados=4)

### 📝 Exercício 2: Análise de Custo

**Contexto:** Você tem 100 PDFs de 50 páginas cada (média de 3000 chars/página).

**Desafio:** Compare duas estratégias:
- **Estratégia A:** chunk_size=500, k=5
- **Estratégia B:** chunk_size=1000, k=3

Qual estratégia envia menos tokens ao LLM em média?

In [43]:
# SEU CÓDIGO AQUI
# Dica: Simule uma página típica e calcule tokens para cada estratégia

# pagina_tipica = "..." * 50  # ~3000 chars
# estrategia_a = analise_completa_rag(pagina_tipica, chunk_size=500, k_recuperados=5)
# estrategia_b = analise_completa_rag(pagina_tipica, chunk_size=1000, k_recuperados=3)

### 📝 Exercício 3: Detecção de Problemas

**Cenário:** Um colega configurou um sistema RAG assim:
- chunk_size=5000
- chunk_overlap=0
- k=1

**Desafio:** Identifique 3 problemas com essa configuração e sugira melhorias.

In [44]:
# SEU CÓDIGO AQUI
# Teste a configuração problemática e compare com uma melhor

# resultado_ruim = analise_completa_rag(artigo, chunk_size=5000, chunk_overlap=0, k_recuperados=1)
# resultado_bom = analise_completa_rag(artigo, chunk_size=???, chunk_overlap=???, k_recuperados=???)

---

## Conclusão

### O que aprendemos:

1. **Tokens** são unidades de processamento do LLM
   - Determinam custos e limites de contexto
   - Variam conforme o modelo/encoding

2. **Chunks** são unidades de armazenamento e busca
   - Determinam precisão da recuperação
   - Você controla o tamanho e overlap

3. **A relação entre eles:**
   ```
   Chunks (sua escolha) → Busca → Tokens (para o LLM)
   ```

4. **Otimização é balanceamento:**
   - Chunks menores = busca precisa, mas menos contexto
   - Chunks maiores = mais contexto, mas busca vaga
   - Tokens recuperados devem caber no limite do modelo

### 🚀 Próximos Passos:

1. Aplique essas calculadoras nos seus próprios documentos
2. Experimente diferentes configurações e meça qualidade das respostas
3. Documente suas melhores configurações para cada tipo de documento
4. Compartilhe descobertas com a turma!

---

**Boa prática em RAG! 🎓**